In [65]:
## web crawling '만개의레시피'

import urllib.request
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
import os
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException

#검색할 RCP_NM list
recipe=pd.read_csv("d:\\data.csv", encoding = 'euc-kr')
name_list=list(recipe["식품명"])
path="C:\\chromedriver.exe"


driver=webdriver.Chrome(path)
driver.maximize_window()


food_dicts=[]
x_data1=[]
for keyword in name_list:
    driver.get('https://www.10000recipe.com/index.html')
    
    # 검색창 : 'RCP_NM'
    driver.find_element_by_id('srhRecipeText').send_keys(keyword)
    driver.find_element_by_class_name('glyphicon-search').click()
    time.sleep(1)
    
    try:
        # 정확성순으로 보기
        ac=driver.find_element_by_class_name('nav>li')
        ac.click()
        recipe=driver.find_element_by_class_name('ea4>li')
        recipe.click()
    except NoSuchElementException:
        continue
    
   # time.sleep(1)

    req=urllib.request.Request(driver.current_url)
    sourcecode=urllib.request.urlopen(driver.current_url).read()
    soup=bs(sourcecode, "html.parser")
    
    recipe_title=[] # RCP_NM
    recipe_source={} # RCP_PARTS_DTLS
    recipe_step=[] # RCP MANUAL
    recipe_img=[] # IMG
    
    # RCP_NM 
    res=soup.find('div', 'view2_summary')
    res=res.find('h3')
    recipe_title.append(res.get_text())
    res=soup.find('div', 'ready_ingre3')

    # RCP_PARTS_DTLS
    try :
        for n in res.find_all('ul'):
            source=[]
            title=n.find('b').get_text()
            recipe_source[title]=''
            for tmp in n.find_all('li'):
                source.append(tmp.get_text().replace(' ','').replace(' ',''))
            recipe_source[title]=source
    except (AttributeError): #검색결과없음 예외처리
        xdata={'음식명': keyword}
        x_data1.append(xdata)
        x_data2=pd.DataFrame(x_data1)
        x_data2.to_csv('D:\\예외.csv', encoding='cp949') 
        pass

    # RCP MANUAL
    try:
        res=soup.find('div','view_step')
        i=0
        for n in res.find_all('div','view_step_cont'):
            i=i+1
            recipe_step.append(str(i)+' '+n.get_text().replace('\n',' '))
    except (AttributeError):
        pass

    # IMG
    res=soup.find('div', 'centeredcrop')
    res=res.find('img')
    recipe_img=res.get('src')
    
    # Data Dictionary
    food_dict={"본이름":keyword, "이름":recipe_title,
              "재료":recipe_source,
              "순서":recipe_step,
              "이미지":recipe_img }
    food_dicts.append(food_dict)

    
    data=pd.DataFrame(food_dicts)   
    data.to_csv('D:\\음식.csv', encoding='cp949') 

    
      